<a href="https://colab.research.google.com/github/esml1101/app5_56/blob/main/2DoF_robot_arm_kinematic_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Robot arm kinematic control

This page introduce the concept of controlling the end-effector of a robot when its joint are controlled with velocity set-points.

## Importing Librairies

In [ ]:
# Loading a robotic toolbox
!git clone https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')
import pyro

# Generic python tools
import numpy as np
from IPython import display

# Defining the robot arm

Here we load a already defined class from the library including the dynamic equations defining the robot arm behavior.



In [ ]:
from pyro.dynamic import manipulator

# Dynamic model (inputs are motor torques)
torque_controlled_robot    = manipulator.TwoLinkManipulator()

torque_controlled_robot.l1 = 4.0 # length of first rigid link
torque_controlled_robot.l2 = 3.0 # length of second rigid link
torque_controlled_robot.l_domain = 10

# Kinematic only model (inputs are motor velocities)
speed_controlled_robot  = manipulator.SpeedControlledManipulator.from_manipulator( torque_controlled_robot )

robot = speed_controlled_robot # For this exercise, we will only use the kinematic model

## Showing the robot behavior with constant velocity inputs

Here we run a simulation of the robot arm with no controllers:

In [ ]:
# robot initial states [ joint 1 angle (rad),  joint 2 angle (rad), joint 1 velocity (rad/sec),  joint 1 velocity (rad/sec)]
robot.x0 = np.array([ 0.1, 0.1])

# robot constant inputs
robot.ubar = np.array([ 0.5, 1.0]) # Constant joint velocities

# run the simulation
robot.compute_trajectory( tf = 6 )

# Animate and display the simulation
ani  = robot.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

# Kinematic controller

Here bellow you can write your own custom kinematic control law, i.e. computing the joint velocity based on actual joint position and time. Here the framework is already set in place for you to speed-up the implementation process. The following variable are pre-computed and available for you to use in the feedback law:

*   The joint angles $q$
*   The time $t$

Furthermore, based on a kinematic robot model the following properties are also pre-computed:
*   The actual end-effector position $r$
*   The Jacobian $J(q)$



In [ ]:
from pyro.control  import robotcontrollers

class CustomKinematicController( robotcontrollers.EndEffectorKinematicController ) :

    #############################
    def c( self , y , r , t ):
        """
        Feedback static computation u = c(y,r,t)

        INPUTS
        y  : sensor signal vector     p x 1
        r  : reference signal vector  k x 1
        t  : time                     1 x 1

        OUTPUTS
        dq  : joint velocity vector   m x 1

        """
        global indice
        # Feedback from sensors
        q = y                  # Joint angles

        # Pre-computed values based on the robot kinematic model
        J = self.J( q )        # Jacobian computation
        r = self.fwd_kin( q )  # End-effector postion

        ##############################
        # YOUR CODE BELLOW !!
        ##############################

        vmax=1
        amax=1

        # pour atteindre la vitesse max sa prend combien de temps
        tmax=1

        #r_d=np.array([5,0])

        dr_d= np.array([0,0])
        if 0<t<=1:
          dr_d = amax*t*np.array([-1.0,0])
        elif 1<t<=2:
          dr_d=np.array([-1.0,0])
        elif 2<t<=3:
          dr_d=amax*(3-t)*np.array([-1.0,0])
        elif 3<t<=4:
          dr_d=amax*(t)*np.array([-0.7,0.7])
        elif 4<t<=8.45:
          dr=np.array([-0.7,0.7])
        elif 8.45<t<=9.45:
          dr_d=amax*(9.45-t)*np.array([-0.7,0.7])






        #r_d = np.array([[5,0],[4,0],[3,0],[0,3],[0,5]])
        #dr_d = np.array([1.0,1.0])
        #e = r_d- r
        #if np.linalg.norm(e)<0.1:
                  #indice+=1

        # Compute the reference
        #r_d  = np.zeros(2) # Place-holder
        #dr_d = np.zeros(2) # Place-holder

        # Compute the desired effector velocity
        #dr_r = np.array([-0.1,0.1]) # Place holder

        sigma = 10
        dr_r = dr_d #+ sigma*(e)

        # From effector speed to joint speed
        dq = np.linalg.inv( J ) @ dr_r

        return dq

In [ ]:
indice=0
ctl = CustomKinematicController( robot )

## Simulating the robot in closed-loop

Here we run a simulation of the robot with the controller.

In [ ]:
# Create the closed-loop system
robot_with_controller = ctl + robot

# Run the simulation
robot_with_controller.x0[0] = 1.0
robot_with_controller.x0[1] = 1.0
robot_with_controller.compute_trajectory( tf = 5 )

In [ ]:
# Plot systems states
robot_with_controller.plot_trajectory('x')

In [ ]:
# Plot control inputs
robot_with_controller.plot_trajectory('u')

In [ ]:
# Animate the simulation
ani = robot_with_controller.generate_simulation_html_video()
html = display.HTML(ani)
display.display(html)